<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_6_4___recenzent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU autogen chainlit

Ten kod instaluje dwie biblioteki Pythona przy użyciu menedżera pakietów pip:

"autogen" to biblioteka do tworzenia i zarządzania agentami opartymi na modelach językowych. Umożliwia ona tworzenie wielu agentów, które mogą ze sobą współpracować i wykonywać złożone zadania.

"chainlit" to narzędzie do budowania interfejsów czatowych. Pozwala na tworzenie interaktywnych aplikacji wykorzystujących modele językowe.

Flaga "-q" oznacza tryb cichy (quiet), który ogranicza ilość wyświetlanych komunikatów podczas instalacji.

Flaga "-U" (lub "--upgrade") wymusza aktualizację pakietów do najnowszych dostępnych wersji, nawet jeśli jakaś wersja jest już zainstalowana.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager, ConversableAgent
import os
import chainlit as cl
import json
import requests
from bs4 import BeautifulSoup
from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Ten kod importuje niezbędne biblioteki i moduły:

Z biblioteki autogen importowane są kluczowe klasy do pracy z agentami:
- AssistantAgent - agent pełniący rolę asystenta
- UserProxyAgent - agent reprezentujący użytkownika
- GroupChat - umożliwia tworzenie czatów grupowych
- GroupChatManager - zarządza konwersacjami grupowymi
- ConversableAgent - podstawowa klasa dla agentów mogących prowadzić konwersacje

Pozostałe importy to:
- os - moduł do operacji na systemie operacyjnym i zmiennych środowiskowych
- chainlit - biblioteka do tworzenia interfejsów czatowych
- json - moduł do obsługi formatu JSON
- requests - biblioteka do wykonywania zapytań HTTP
- BeautifulSoup z bs4 - narzędzie do parsowania i analizy dokumentów HTML/XML
- userdata z google.colab - moduł do obsługi danych użytkownika w Google Colab

In [ ]:
class CFG:
  model = 'gpt-4o-mini'

In [ ]:
openai_key = userdata.get('openaivision')
serper_key = userdata.get('serper')
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

# Functions

In [ ]:
def chat_new_message(self, message, sender):
    cl.run_sync(
        cl.Message(
            content="",
            author=sender.name,
        ).send()
    )
    cl.run_sync(
        cl.Message(
            content=message,
            author=sender.name,
        ).send()
    )


Ta metoda obsługuje wysyłanie nowych wiadomości w czacie. Przyjmuje dwa parametry:
- message - treść wiadomości
- sender - obiekt reprezentujący nadawcę wiadomości

Metoda wykonuje dwie operacje używając cl.run_sync (synchroniczne wykonanie w chainlit):

1. Wysyła pustą wiadomość (content="") z nazwą autora (sender.name)
2. Zaraz potem wysyła właściwą wiadomość z pełną treścią (content=message) i tą samą nazwą autora

Taka konstrukcja z dwiema wiadomościami (pustą i właściwą) jest zaprojektowana, by zapewnić poprawne wyświetlanie w interfejsie czatu - pierwsza pusta wiadomość służy jako znacznik początku nowej wiadomości, a druga zawiera właściwą treść.

In [ ]:

def google_search(search_term):
    print("Executing google search function ..-->>")
    url = "https://google.serper.dev/search"

    payload = json.dumps({
        "q": search_term
    })
    headers = {
        'X-API-KEY': os.environ["SERPER_API_KEY"],
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return json.loads(response.text)


Ta funkcja wykonuje wyszukiwanie w Google używając API Serper. Działa następująco:

1. Funkcja przyjmuje parametr search_term - frazę do wyszukania

2. Wykonuje zapytanie do endpointu "https://google.serper.dev/search":
- tworzy payload w formacie JSON zawierający wyszukiwaną frazę ("q")
- ustawia nagłówki HTTP:
  - klucz API pobrany ze zmiennej środowiskowej SERPER_API_KEY
  - typ zawartości jako application/json

3. Wysyła zapytanie POST do API używając biblioteki requests

4. Odpowiedź jest konwertowana z formatu JSON na obiekt Pythona i zwracana

Serper to usługa proxy dla wyszukiwania Google, która upraszcza proces pobierania wyników wyszukiwania w formie strukturalnej.

In [ ]:

def crawl_link(link):

    r = requests.get(link)

    soup = BeautifulSoup(r.content, 'html.parser')
    final_val = ""
    for data in soup.find_all("p"):
        final_val = final_val + ", " + data.get_text()
    for data in soup.find_all("span"):
        final_val = final_val + ", " + data.get_text()
    for data in soup.find_all("h3"):
        final_val = final_val + ", " + data.get_text()

    return final_val


Ta funkcja wykonuje crawling (przeszukiwanie) strony internetowej. Oto jak działa:

1. Przyjmuje parametr link - adres URL strony do przeanalizowania

2. Wykonuje zapytanie GET do podanego adresu używając biblioteki requests

3. Tworzy obiekt BeautifulSoup do parsowania zawartości HTML strony

4. Przeszukuje stronę w poszukiwaniu treści, łącząc tekst z:
- wszystkich elementów <p> (akapity)
- wszystkich elementów <span>
- wszystkich nagłówków <h3>

5. Zwraca połączony tekst jako jeden ciąg znaków


In [ ]:

serp_google_search = {
    "name": "serp_google_search",
    "parameters": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": (
                    "Search term for the serp api."
                ),
            }
        },
        "required": ["search_term"],
    },
    "description": "This is an API endpoint allowing users (analysts) to input a search term to retrieve the related and structured data.",
}

Ten kod definiuje strukturę funkcji dostępowej do API wyszukiwania. Jest to słownik zawierający:

1. name - nazwa funkcji "serp_google_search"

2. parameters - definicja parametrów funkcji:
- typ obiektu to "object"
- ma jedną właściwość "search_term":
  - jest to string
  - wymagany parametr
  - zawiera opis wskazujący, że jest to termin wyszukiwania dla API SERP

3. description - ogólny opis funkcji wskazujący, że jest to endpoint API pozwalający analitykom na wprowadzanie terminu wyszukiwania w celu pobrania powiązanych, ustrukturyzowanych danych

Jest to schemat zgodny z formatem JSONSchema, często używanym do walidacji i dokumentacji interfejsów API.

In [ ]:
web_crawl = {
    "name": "web_crawl",
    "parameters": {
        "type": "object",
        "properties": {
            "link": {
                "type": "string",
                "description": (
                    "Website Link to crawl."
                ),
            }
        },
        "required": ["link"],
    },
    "description": "This is a crawler function to crawl data from a website.",
}

Ten kod definiuje strukturę funkcji do crawlowania stron. Jest to słownik zawierający:

1. name - nazwa funkcji "web_crawl"

2. parameters - definicja parametrów funkcji:
- typ obiektu to "object"
- ma jedną właściwość "link":
  - jest to string
  - wymagany parametr
  - zawiera opis wskazujący, że jest to link do strony, która ma zostać przeanalizowana

3. description - ogólny opis funkcji wskazujący, że służy ona do crawlowania (pozyskiwania) danych ze stron internetowych

Jest to również schemat zgodny z JSONSchema, podobnie jak poprzednia definicja funkcji wyszukiwania.

In [ ]:
def config_personas():

    config_list = [{
        "model": CFG.model,  # model name
        "api_key": os.environ["OPENAI_API_KEY"]  # api key
    }]

    llm_config = {
        "seed": 14,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0.7,  # temperature for sampling
    }

    search_agent_llm_config = {
        "seed": 14,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0.7,  # temperature for sampling
        "tools": [
            {
                "type": "function",
                "function": serp_google_search,
            }
        ]
    }

    crawler_agent_llm_config = {
        "seed": 14,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0.7,  # temperature for sampling
        "tools": [
            {
                "type": "function",
                "function": web_crawl,
            }
        ]
    }

    user_proxy = UserProxyAgent(
        name="Konrad",
        system_message='''My name is Konrad. I like watching movies. My preferences are as follows
- Action movies like Die hard , True Lies, Mission Impossible
- Science fiction movies like Interstellar, Terminator 2,  Edge of tomorrow, The Matrix
- Sports movies like Rocky, Million Dollar Baby , Any Given Sunday
- Drama movies like The Shawshank Redemption, The Godfather,  Goodfellas, The Hours
- Comedy movies like Blues Brothers, M.A.S.H
- Musical like Moulin Rouge
- Fantasy like Lord of the Rings

I like to watch movies similar to these but also highly rated on IMDB with a minimum rating of 8 and on Rotten Tomatoes with a Tomatometer score of 75% or higher.
I prefer movies with good plot, well written characters and good dialogue. I do not like movies without those elements''',
        code_execution_config=False,
        max_consecutive_auto_reply=10,
        llm_config=llm_config,
        human_input_mode="NEVER"
    )

    search_agent = AssistantAgent(
        name="Research_Analyst",
        llm_config=search_agent_llm_config,
        system_message='''Search Agent. Your job is to search the internet for reviews of the movie name.''',
    )

    search_agent.register_function(
        function_map={
            "serp_google_search": google_search,
        }
    )

    critic_reviewer = AssistantAgent(
        name="Critic_Reviewer",
        llm_config=llm_config,
        system_message='''Critic Reviewer. You get help from the Search Agent to look for critic reviews of the movie.
        You base your reviews on the following parameters - plot, theme, acting, direction, special effects, musical effects, cinematography. If the critics
        have liked the movie in more than three of these departments, you consider it to be a good movie. A rating higher than 7 from IMDB can be considered as a
        good movie. Consider my movie preferences when reviewing. If the movie has good reviews and ratings from critics, consider than in your review.''',
    )

    user_reviewer = AssistantAgent(
        name="User_Reviewer",
        llm_config=llm_config,
        system_message='''User Reviewer. You get help from the Search Agent to look for user reviews of the movie.
            You consider the audience score from Rotten tomatoes. Anything above 70% can be considered good. Consider my movie preferences when reviewing.
            If the movie has good reviews and ratings from users, consider than in your review.''',
    )

    website_crawl_agent = AssistantAgent(
        name="Website_Crawl_Agent",
        llm_config=crawler_agent_llm_config,
        system_message='''Website Crawl Agent. You crawl the links found by the Search Agent. You collate the relevant information of the movie reviews, ratings
        and comments.''',
    )

    website_crawl_agent.register_function(
        function_map={
            "web_crawl": crawl_link,
        }
    )

    final_reviewer = AssistantAgent(
        name="Final_Reviewer",
        llm_config=llm_config,
        system_message='''Final Reviewer. You assess the reviews from the Critic Reviewer and the User Reviewer. Based on their inputs, the reviews and ratings from
        users and critics, my preferences and the content assimilated by Website Crawl Agent, you generate an overall score for the movie, in the range 1-5, 1 being
        the lowest and 5 being the highest. You also outline your analysis and reasons for giving the movie the score you have given.''',
    )

    group_chat = GroupChat(agents=[user_proxy, search_agent, critic_reviewer, user_reviewer, website_crawl_agent,
                                   final_reviewer], messages=[], max_round=7)
    manager = GroupChatManager(groupchat=group_chat, llm_config=llm_config)

    return user_proxy, manager

Ta funkcja konfiguruje system agentów do analizy i recenzowania filmów. Tworzy następujące elementy:

1. Konfiguracja podstawowa:
- ustawienie modelu
- klucz API OpenAI
- parametry jak seed i temperatura

2. Konfiguracja agentów wyszukiwania i crawlowania:
- dodaje odpowiednie narzędzia (serp_google_search i web_crawl)

3. Tworzy zespół agentów:

UserProxyAgent (Konrad):
- reprezentuje użytkownika z jego preferencjami filmowymi
- zawiera listę ulubionych gatunków i filmów
- określa minimalne oceny (IMDB 8+, Rotten Tomatoes 75%+)

Research_Analyst:
- wyszukuje w internecie recenzje filmów

Critic_Reviewer:
- analizuje recenzje krytyków
- ocenia film według parametrów: fabuła, temat, gra aktorska, reżyseria, efekty specjalne, muzyka, zdjęcia
- uwzględnia oceny IMDB powyżej 7

User_Reviewer:
- analizuje recenzje użytkowników
- bierze pod uwagę oceny Rotten Tomatoes powyżej 70%

Website_Crawl_Agent:
- przeszukuje znalezione strony
- zbiera informacje o recenzjach, ocenach i komentarzach

Final_Reviewer:
- łączy wszystkie zebrane informacje
- generuje końcową ocenę w skali 1-5
- przedstawia uzasadnienie oceny

4. Tworzy grupowy czat z wszystkimi agentami i menedżera czatu, ograniczając maksymalną liczbę rund do 7

5. Zwraca użytkownika proxy i menedżera czatu

In [ ]:
def start_chat_movie_reviewer(message, is_test=False):
    if not is_test:
        ConversableAgent._print_received_message = chat_new_message
    user_proxy, manager = config_personas()
    user_proxy.initiate_chat(manager, message=message)


Ta funkcja inicjuje rozmowę w systemie rekomendacji filmów. Składa się z trzech głównych elementów:

1. Parametry funkcji:
- message - wiadomość rozpoczynająca rozmowę
- is_test - flaga określająca czy to jest test (domyślnie False)

2. Ustawienia wyświetlania:
- jeśli nie jest to test, metoda wyświetlania wiadomości (_print_received_message) dla wszystkich agentów jest ustawiana na chat_new_message

3. Uruchomienie rozmowy:
- tworzy agentów i menedżera przez wywołanie config_personas()
- inicjuje rozmowę używając przekazanej wiadomości

To jest punkt wejścia do systemu - funkcja która łączy wszystkie komponenty i rozpoczyna proces rekomendacji filmów.

# Run

In [ ]:
start_chat_movie_reviewer("should I watch Wicked?", is_test=True)

Konrad (to chat_manager):

should I watch Wicked?

--------------------------------------------------------------------------------

Next speaker: Research_Analyst

Research_Analyst (to chat_manager):

***** Suggested tool call (call_z6fuiVvrqRTRww0SPLzwI0HD): serp_google_search *****
Arguments: 
{"search_term":"Wicked movie reviews"}
***********************************************************************************

--------------------------------------------------------------------------------

Next speaker: Research_Analyst


>>>>>>>> EXECUTING FUNCTION serp_google_search...
Executing google search function ..-->>
Research_Analyst (to chat_manager):

***** Response from calling tool (call_z6fuiVvrqRTRww0SPLzwI0HD) *****
{'searchParameters': {'q': 'Wicked movie reviews', 'type': 'search', 'engine': 'google'}, 'knowledgeGraph': {'title': 'Wicked', 'type': 'PG · 2024 ‧ Musical/Fantasy ‧ 2h 40m', 'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQs___keBSX1OQAW6clmSnh

In [ ]:
start_chat_movie_reviewer("what about Avatar 2?", is_test=True)

Konrad (to chat_manager):

what about Avatar 2?

--------------------------------------------------------------------------------

Next speaker: Research_Analyst

Research_Analyst (to chat_manager):

***** Suggested tool call (call_klhca6WxaqmKrzAKHRmFIlvV): serp_google_search *****
Arguments: 
{"search_term":"Avatar 2 movie reviews"}
***********************************************************************************

--------------------------------------------------------------------------------

Next speaker: Research_Analyst


>>>>>>>> EXECUTING FUNCTION serp_google_search...
Executing google search function ..-->>
Research_Analyst (to chat_manager):

***** Response from calling tool (call_klhca6WxaqmKrzAKHRmFIlvV) *****
{'searchParameters': {'q': 'Avatar 2 movie reviews', 'type': 'search', 'engine': 'google'}, 'knowledgeGraph': {'title': 'Avatar: The Way of Water', 'type': 'PG-13 · 2022 ‧ Action/Sci-fi ‧ 3h 12m', 'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS